<a href="https://colab.research.google.com/github/bxck75/piss-ant-pix2pix/blob/master/modeltransferv2_apes_imshow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# remove defaults
!rm -r sample_data
# Clone the repo
!git clone https://github.com/bxck75/Python_Helpers.git

# Change dir
%cd /content/Python_Helpers
# install
!python setup.py install

from IPython.display import clear_output
import main
import os
import sys
import IPython
import Helpers
P=Helpers.core.Core()
hlp=P.H
clear_output()

In [0]:
# get repo
%cd /content/
hlp.repolist= hlp.repo_collection
repos_sorted = hlp.repo_collection.repos_sorted
A1=['bxck75/piss-ant-pix2pix']
hlp.Me(['inst_reps',A1,'/content/installed_repos',False,True])
clear_output()

In [0]:
# get images
%cd /content/

'''insects'''
# images_set_name='insect'
# images_set_code='1fbEUcBIvLwjlVFrUiW2g8KtRTvFSsW9D'
# images_model_code='1eFR5ZHfp5PzPEnFv_v3xxE9T0tKPJTS4'
'''garbagepailkids'''
# images_set_name='garbagepailkids'
# images_set_code='1nb8yr9TIJaiB1GFN9S4mD5GfTNvEQpW-'
# images_model_code='1tp8Gm1fMuSQoLEEXQ_C1Dlw1N4wwFxKb'
'''apes'''
images_set_name='apes'
images_set_code='1PUEpLgRfKKDvaMAVtKBVr-Z0ThQvOGWv'
images_model_code='1vLmCOaActAAZ0bHdrv0YgP-rCnYHX0O9'

'''fetch and unzip'''
Helpers.GdriveD.GdriveD(images_set_code, '/content/'+images_set_name+'.zip')
!unzip /content/{images_set_name}.zip
clear_output()

In [0]:
# make dataset
%cd /content/installed_repos/piss-ant-pix2pix
!bash image_folder_make_set_train_colab.sh /content/{images_set_name} 1 prep
clear_output()

In [0]:
# ''' 
#     startup loop  1x
#     # 1 get metric from gdrive
#     2 train 5 epochs
#     3 dump metrics to gdrive with the same file id 
# '''
# for i in range(1):
#     # fetch metrics
#     %cd /content/
#     # Helpers.GdriveD.GdriveD(images_model_code, '/content/'+images_set_name+'_model_in.zip')
#     # !unzip /content/{images_set_name}_model_in.zip -d metrics/
#     # !rm -r /content/metrics/options.json
#     # train epochs
#     %cd /content/installed_repos/piss-ant-pix2pix
#     !python pix2pix.py --output_dir /content/metrics --progress_freq 10 --save_freq 50 --summary_freq 50 --display_freq 200 --max_epochs 5 --mode train --input_dir /content/{images_set_name}images/_combined/train --which_direction 'BtoA'
#     # dump metrics
#     %cd /content/
#     Helpers.Core()
#     obj=Helpers.core.Core()
#     folder_of_model='metrics'
#     result=obj.H.zip(images_set_name+'model',obj.Gdrive_root+'/models',folder_of_model).ZipUp
#     zip_hash=result.split('(id) ')[1]
#     print(zip_hash)
#     # delete old crap
#     !rm -r /content/metrics
#     !rm -r /content/{images_set_name}model.zip

start values
progress  epoch 1  step 10  image/sec 0.7  remaining 122m
dloss_GAN 0.11989411
gloss_GAN 0.060013566
gloss_L1 0.043864187

In [0]:
import sys
import os
import dlib
import glob
%matplotlib inline
from matplotlib import pyplot as plt
import cv2
''' 
    loop  10x
    1 get metric from gdrive
    2 train 5 epochs
    3 dump metrics to gdrive with the same file id 
'''
for i in range(10):
    ''' fetch metrics'''
    %cd /content/
    Helpers.GdriveD.GdriveD(images_model_code, '/content/'+images_set_name+'_model_in.zip')
    !unzip /content/{images_set_name}_model_in.zip -d metrics/

    ''' train epochs '''
    %cd /content/installed_repos/piss-ant-pix2pix
    !python pix2pix.py --checkpoint /content/metrics --output_dir /content/metrics --progress_freq 110 --save_freq 150 --summary_freq 0 --display_freq 100 --max_epochs 5 --mode train --input_dir /content/{images_set_name}images/_combined/train --which_direction 'BtoA'

    %cd /content/
    '''clean old models,test images,logs etc befor zipping'''
    # delete unwanted
    !rm -r /content/metrics/index.html

    # clean up images
    img_list=hlp.Me(['globx','/content/metrics/images','*.*g'])
    img_list = sorted(img_list)
    print(img_list)
    n=6 #pop the last 6 items off the list (latest images)
    latest = img_list[-n:]
    for i in range(n):
        img = dlib.load_rgb_image(latest[i]) 
        plt.imshow(img)
    del img_list[-n:]
    # delete the files left in the list
    for i_file in img_list:
        img = dlib.load_rgb_image(i_file) 
        plt.imshow(img)
        print('deleting : ' + i_file)
        !rm -r {i_file}
    plt.show()

    # clean up models
    models_list=hlp.Me(['globx','/content/metrics','model-*'])
    models_list = sorted(models_list)
    print(models_list)
    n=3 #pop the last 3 items off the list (latest model)
    del models_list[-n:]
    # delete the files left in the list
    for m_file in models_list:
        print('deleting : ' + m_file)
        !rm -r {m_file}
    # clean up events
    events_list=hlp.Me(['globx','/content/metrics','events*'])
    events_list = sorted(events_list)
    print(events_list)
    n=1 #pop the last item off the list (latest event) 
    del events_list[-n:]
    # delete the files left in the list
    for e_file in events_list:
        print('deleting : ' + e_file)
        !rm -r {e_file}

    ''' zip metrics up to gdrive '''
    %cd /content/
    Helpers.Core()
    obj=Helpers.core.Core()
    folder_of_model='metrics'
    result=obj.H.zip(images_set_name+'model',obj.Gdrive_root+'/models',folder_of_model).ZipUp
    zip_hash=result.split('(id) ')[1]
    print(zip_hash)

    # delete old metrics before unpacking new in the beginning of the loop
    !rm -r /content/metrics
    !rm -r /content/{images_set_name}model.zip

In [0]:
# # rewrite checkpointfile
# !echo 'model_checkpoint_path: "/content/metrics/model-21880"' > /content/metrics/checkpoint
# !echo 'all_model_checkpoint_paths: "/content/metrics/model-21880"' >> /content/metrics/checkpoint

In [0]:
# # download/unzip metrics 
# Get pretrained metrics
# %cd /content/
# Helpers.GdriveD.GdriveD(images_model_code, '/content/'+images_set_name+'_model_in.zip')
# !unzip /content/{images_set_name}_model_in.zip -d metrics/

In [0]:
# # remove old crap before rezipping
# !rm -r /content/metrics/images
# !rm -r /content/metrics/index.html

# clean up models
models_list=hlp.Me(['globx','/content/metrics','model-*'])
models_list = sorted(models_list)
print(models_list)
n=3 #pop the last 3 items off the list (latest model)
del models_list[-n:]
for m_file in models_list:
    print('deleting : ' + m_file)
    !rm -r {m_file}

# clean up events
events_list=hlp.Me(['globx','/content/metrics','events*'])
events_list = sorted(events_list)
print(events_list)
n=1 #pop the last 1 items off the list (latest event)
del events_list[-n:]
for e_file in events_list:
    print('deleting : ' + e_file)
    !rm -r {e_file}

In [0]:
# # rezip
# %cd /content/
# Helpers.Core()
# obj=Helpers.core.Core()
# folder_of_model='metrics'
# print(obj.Gdrive_root+'/models')
# result=obj.H.zip(images_set_name+'model',obj.Gdrive_root+'/models',folder_of_model).ZipUp
# # print(result)
# zip_hash=result.split('(id) ')[1]
# print(zip_hash)

In [0]:
# test the current checkpoint
# get metrics
%cd /content/
Helpers.GdriveD.GdriveD(images_model_code, '/content/'+images_set_name+'_model_in.zip')
!unzip /content/{images_set_name}_model_in.zip -d metrics/
# test 
%cd /content/installed_repos/piss-ant-pix2pix
!python pix2pix.py --mode test --output_dir /content/{images_set_name}_pretrained/test --input_dir /content/{images_set_name}images/_combined/val --max_steps 100 --which_direction "BtoA" --seed 0 --checkpoint /content/metrics
# export
!python pix2pix.py --mode export --output_dir /content/{images_set_name}_pretrained/exp --checkpoint /content/metrics


In [0]:
# /content/apes_pretrained/exp
# rezip
%cd /content/
Helpers.Core()
obj=Helpers.core.Core()
folder_of_model=images_set_name+'_pretrained/exp'
print(obj.Gdrive_root+'/models')
result=obj.H.zip(images_set_name+'model_exp',obj.Gdrive_root+'/models',folder_of_model).ZipUp
# print(result)
zip_hash=result.split('(id) ')[1]
print(zip_hash)

In [0]:
# Example python rename.py '/root/Bureaublad/data/boefjes/front' 
def rename(directory_in,directory_out):    
    from PIL import Image
    import os,sys,glob
    # directory=sys.argv[1]
    i=int(0)
    for infilename in glob.iglob(directory_in+'/*.*g'):
        im = Image.open(infilename)
        rgb_im = im.convert('RGB')
        outfilename = "/img%d.png" % int(i + 1)
        outfile=os.path.join(directory_out, outfilename)
        print(directory_out+outfile)
        rgb_im.save(directory_out+outfile)
        i += 1
        
 edged_images = glob.iglob('/content/metrics/images/*inputs.png')
 predicted_images = glob.iglob('/content/metrics/images/*outputs.png')
 original_images = glob.iglob('/content/metrics/images/*targets.png')

# zipping lists of lists  
# using map() + __add__ 
import itertools 
  
# initializing lists 
test_list1 = [[1, 3], [4, 5], [5, 6]] 
test_list2 = [[7, 9], [3, 2], [3, 10]] 
  
# printing original lists 
print ("The original list 1 is : " + str(test_list1)) 
print ("The original list 2 is : " + str(test_list2)) 
  
# using map() + __add__ 
# zipping lists of lists  
res = [list(itertools.chain(*i))  
       for i in zip(test_list1, test_list2)] 
      
# printing result  
print ("The modified zipped list is : " +  str(res))